In [ ]:
import os
!pip install faiss-cpu sentence-transformers
# !pip install pinecone-client sentence-transformers
# !pip uninstall pinecone-client pinecone -y
!pip install pinecone

In [ ]:
import re
def preprocess_text(text):
    """Cleans text by removing special characters and extra spaces."""
    text = text.lower()  # Convert to lowercase
    text = re.sub(r"[^a-zA-Z0-9.,!?'\n]", " ", text)  # Remove special characters except punctuation
    text = re.sub(r"\s+", " ", text).strip()  # Remove extra spaces
    return text

# Apply preprocessing
clean_stories = [preprocess_text(story) for story in stories]

print("Sample Cleaned Story:\n", clean_stories[0])

Sample Cleaned Story:
 caroline. i learned her name through the phonebook, my shaking fingers carefully caressing its pages as i searched for the address i'd seen her at so many times. 43 mako drive, the small, brick house on the corner of braxton and mako. i'd memorized the shape of her home weeks before, my bare feet sliding across its wet grass every time i closed my eyes. letters from her mailbox, addressed to caroline smith, confirmed what the book claimed. she was perfect, absolutely flawless in every way. i'd watched her for seven months almost every single day and night silently following her as she strolled to and from her classes. sometimes i stayed outside her bedroom window as we slept, my heart racing as i matched my breathing to hers. she never knew i was there, never acknowledged me as more than the distant shadow of a faceless tree, but i knew she needed me. she was all i could think about, all i wanted to be with. beautiful, flawless, ideal. if anything could convince 

In [ ]:
import os
from pinecone import Pinecone, ServerlessSpec

# Set API Key
PINECONE_API_KEY = "" # hidden for github upload

# Initialize Pinecone
pc = Pinecone(api_key=PINECONE_API_KEY)

# Define index name
index_name = "story-snippets"

# Check available indexes
existing_indexes = pc.list_indexes().names()
print("Existing Indexes:", existing_indexes)

# If index does not exist, create it in a supported region
if index_name not in existing_indexes:
    pc.create_index(
        name=index_name,
        dimension=384,
        metric="cosine",
   spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    )
    )
    print(f"Index '{index_name}' created successfully!")

# Connect to the index
index = pc.Index(index_name)
print(f"Pinecone index '{index_name}' is ready!")



Existing Indexes: ['story-snippets']
Pinecone index 'story-snippets' is ready!


In [ ]:
import numpy as np
from sentence_transformers import SentenceTransformer
from pinecone import Pinecone, ServerlessSpec

# Load the Sentence Transformer model
model = SentenceTransformer("all-MiniLM-L6-v2")


def embed_stories(stories):
    """Converts stories into vector embeddings using SentenceTransformer."""
    batch_size = 100  # Reduce batch size
    embeddings = []  # Store all embeddings

    for i in range(0, len(stories), batch_size):
        batch = stories[i : i + batch_size]
        batch_embeddings = model.encode(batch, convert_to_numpy=True)  # Embed batch
        embeddings.extend(batch_embeddings)  # Append batch embeddings to the list

    return embeddings

# Convert all stories into embeddings
# story_embeddings = embed_stories(clean_stories)


# # Store embeddings in Pinecone in smaller batches
# batch_size = 50
# to_upsert = []
# for i, (story, embedding) in enumerate(zip(clean_stories, story_embeddings)):
#     to_upsert.append((f"story_{i}", embedding.tolist(), {"text": story}))  # Store ID, vector, and text metadata

#     # Upload in batches to avoid Pinecone's request size limit
#     if len(to_upsert) >= batch_size or i == len(clean_stories) - 1:
#         index.upsert(vectors=to_upsert)
#         to_upsert = []  # Clear batch

# print(f"Stored {len(clean_stories)} stories in Pinecone.")

# Function to retrieve the most relevant snippet
# def retrieve_snippet(query, top_k=3, sentence_count=2):
#     """Finds the most relevant story snippet given a query using Pinecone."""
#     query_embedding = model.encode([query], convert_to_numpy=True).tolist()  # Convert query to embedding

#     # Corrected query format
#     results = index.query(vector=query_embedding[0], top_k=top_k, include_metadata=True)  # Use query_embedding[0]

#     if results["matches"]:
#         retrieved_story = results["matches"][0]["metadata"]["text"]  # Get full retrieved story
#         sentences = retrieved_story.split(". ")  # Split into sentences
#         snippet = ". ".join(sentences[:sentence_count])  # Return only the first 1-2 sentences
#         return snippet
#     else:
#         return "No matching snippet found."

# # Example: Retrieve a short snippet
# user_input = "nuclear war or other catastrophic event."
# retrieved_snippet = retrieve_snippet(user_input)

# print("\nRetrieved Snippet:\n", retrieved_snippet)



Retrieved Snippet:
 october 23rd, 2027. the day the world was supposed to end


In [ ]:
import numpy as np
from sentence_transformers import SentenceTransformer
import os
from pinecone import Pinecone, ServerlessSpec

# Load pre-trained Sentence Transformer model
model = SentenceTransformer("all-MiniLM-L6-v2")

def retrieve_top_sentences(query):
    """Finds the most relevant sentence and includes the next two sentences for context."""
    query_embedding = model.encode([query], convert_to_numpy=True).tolist()
    results = index.query(vector=query_embedding, top_k=1, include_metadata=True)

    if results["matches"]:
        retrieved_story = results["matches"][0]["metadata"]["text"]
        sentences = retrieved_story.split('. ')  # Split story into sentences

        # Compute similarity between query and each sentence
        sentence_embeddings = model.encode(sentences, convert_to_numpy=True)
        query_embedding = np.array(query_embedding).reshape(1, -1)

        similarities = np.dot(sentence_embeddings, query_embedding.T).flatten()  # Cosine similarity

        # Find the most similar sentence
        most_similar_idx = np.argmax(similarities)

        # Retrieve the most similar sentence + the next two sentences
        start_idx = most_similar_idx
        end_idx = min(start_idx + 3, len(sentences))  # Ensure we don't go out of bounds
        top_sentences = sentences[start_idx:end_idx]

        return top_sentences
    else:
        return ["No matching snippet found."]

# Example usage
user_input = "nuclear war or other catastrophic event."
top_sentences = retrieve_top_sentences(user_input)

print("\nRetrieved Sentences:")
for i, sentence in enumerate(top_sentences, 1):
    print(f"{sentence}")



Retrieved Sentences:
and, in one worldwide motion, hundreds of missiles hit the ground, causing craters and destruction
but not destruction of the expected scale
after a few hours of confusion, people across the world came to the conclusion that something had gone wrong with the missiles


In [ ]:
!pip install google-generativeai

In [ ]:
import google.generativeai as genai

# Set your Gemini API key
GENAI_API_KEY = "" # Hidden for github upload
genai.configure(api_key=GENAI_API_KEY)

# models = genai.list_models()
# for model in models:
#     print(model.name)

# Define a function to query Gemini with RAG snippets
# def query_gemini_story(user_query, retrieved_snippet):
#     """Generates a story continuation using the user query as the opening line, with retrieved snippets for context."""
#     Gemini_model = genai.GenerativeModel("models/gemini-1.5-flash-latest")

#     # Structured prompt for better story generation
#     full_prompt = (
#         f"You are a creative storyteller. Continue the story with 2 more lines, keeping it engaging and immersive.\n\n"
#         f"Opening Line:\n{user_query}\n\n"
#         f"Additional Context:\n{retrieved_snippet}\n\n"
#         f"Continue the story from this point, maintaining a coherent and engaging flow."
#     )

#     response = Gemini_model.generate_content(full_prompt)
#     return response.text


def query_gemini_story(story, retrieved_snippet):
    """Generates a continuation for a story using the given story and retrieved snippet as context.
    Ensures grammatical accuracy, logical continuity, and proper structure.
    If the story exceeds 500 characters, it guides it toward a natural conclusion,
    ending with 'THE END' if appropriate.
    The AI should generate only new content and not repeat existing text.
    """

    Gemini_model = genai.GenerativeModel("models/gemini-2.0-flash")

    # Check if the story should begin wrapping up
    should_end = len(story) > 500

    # Construct a structured prompt for better story continuation
    full_prompt = (
        "You are a skilled storyteller. Your task is to continue the given story  by adding one more, next sentence to the story while maintaining "
        "grammatical accuracy, logical flow, and a consistent narrative style. "
        "Do not repeat any existing content.\n\n"
        f"### Story So Far:\n{story}\n\n"
        f"### Contextual Inspiration:\n{retrieved_snippet}\n\n"
        "Use the above inspiration as guidance but do not copy it. Ensure that the story flows naturally, "
        "maintaining the established tone and style.\n\n"
    )

    if should_end:
        full_prompt += (
            "The story has exceeded 500 characters, so begin guiding it toward a satisfying conclusion. "
            "If a natural ending is possible, conclude with 'THE END.' Otherwise, continue progressing the "
            "narrative while preparing for an eventual resolution."
        )
    else:
        full_prompt += "Continue the story in a coherent and engaging manner."

    # Generate story continuation
    response = Gemini_model.generate_content(full_prompt)

    return response.text.strip()


def format_story_with_ai(story_text):
    """Formats a story properly using a generative AI, ensuring correct paragraph structure, spacing, and readability."""

    Gemini_model = genai.GenerativeModel("models/gemini-1.5-flash-latest")

    prompt = (
        "You are an expert at formatting stories for readability. "
        "Take the following story and format it properly, ensuring:\n"
        "- Correct paragraph breaks.\n"
        "- Proper indentation or spacing.\n"
        "- Clear dialogue formatting with quotes on new lines.\n"
        "- Consistent punctuation and capitalization.\n"
        "- A polished and professional appearance.\n\n"
        "Here is the story:\n\n"
        f"{story_text}\n\n"
        "Please return only the formatted story without adding explanations or extra commentary."
    )

    response = Gemini_model.generate_content(prompt)
    return response.text.strip()


# Example usage
user_query = "He was running as fast as he could."
retrieved_snippet = retrieve_top_sentences(user_query)  # Assuming this function fetches relevant context
print("Retrieved: ", retrieved_snippet)

story_continuation = query_gemini_story(user_query, retrieved_snippet)
print(story_continuation)

formatted_adi_story = format_story_with_ai("It was a usual day. Adi just got out of bed.A tiny, insistent buzzing vibrated against his cheek.He thought it was just his cheek muscles being tired from sleeping in a weird position.But the buzzing persisted, growing louder, a frantic, insistent hum that seemed to burrow into his very skull.It felt like it was not coming from the outside of his cheek, but rather from the inside.He cautiously touched his cheek, his fingers encountering a small, metallic object embedded just beneath the surface of his skin.Suddenly he realised, he's no longer himself. He was a robot.Panic, cold and sharp, sliced through the manufactured calm of his robotic mind.He runs to the bathroom, looking for a mirror, trying to see what sort of monstrosity he had become.His reflection stared back, not the familiar face he expected, but a polished chrome skull, gleaming under the harsh bathroom light, its surface subtly marred by a faint, buzzing tremor.He starts running in panic.His chrome feet slipped on the wet tiles, the metallic shriek echoing the frantic drumming in his artificial heart.And then suddenly he woke up, his phone buzzing against his cheek after he fell asleep talking on his phone. \"Honey,\" she whispered, her voice raspy with sleep, \"are you having a nightmare?\"He sees her and all the panic quickly leaves his body, a smile forming on his face.The smile faltered, replaced by a horrified grimace as he saw the chrome glint of his own hand reflected in her wide, tear-filled eyes.THE END.Then everything went black.")
print(formatted_adi_story)

Retrieved:  it was evening rush hour, and the interstate was clogged worse than a yeti's bathtub. j. robert harris sat in driver's seat of his mercedes and alternately debated the merits of upgrading to this year's model of the car, or just chucking it all in and going to live in a hut of a tropical beach and surviving off fish and coconuts. it is worth noting that robert harris did not particularly like the taste of fish or coconut. it amazed him that this many cars could fit on the highway. when you thought about it, the commute from his law office to his large house in the suburbs was only a few miles, and yet the highway seemed to stretch on interminably. find alternate routes, he ordered. the gps pinged back at him, and said. one alternate route found. it's faster. much, much faster. in fact you'd be an idiot not to take it, a complete moron ouch, there was a muffled thump and a pause, so in four hundred yards turn left. robert harris frowned. was the gps acting up? perhaps he sho

ReadTimeout: HTTPConnectionPool(host='localhost', port=43081): Read timed out. (read timeout=600.0)

In [ ]:
# Start interactive story session
print("Welcome to the AI-assisted storytelling! Type a sentence to begin.")
story = ""
while True:
    user_input = input("You: ")
    if user_input.strip().upper() == "THE END":
        print("Story complete.")
        break
    retrieved_snippet = retrieve_top_sentences(user_input)
    story += user_input
    print(f"Story: {story}\n\nSnippet: {retrieved_snippet}")
    ai_response = query_gemini_story(story, retrieved_snippet)
    story += ai_response
    print(f"AI: {ai_response}")

print("Final Story:")
print(format_story_with_ai(story))

Welcome to the AI-assisted storytelling! Type a sentence to begin.
You: Once upon a time
Story: Once upon a time

Snippet: this story begins not once upon a time on the contrary, it takes place at a specific time, in a specific place, and features two very specific people. in reference to the cosmic calendar, wherein hundreds of years are compressed into the space of a single grain of sand, a day is the most arbitrary of measures. a raindrop in an ocean. however, for this story atleast, a day had all the meaning of a thousand years. it was thursday, march 26th. this was important, you see, because it was to be marshall henry morrison's last day on the planet earth. amy dane had been hesitant to go to that specific place, the greenwich federal prison, but in the end she decided that she must. for even though many years had passed since last she'd seen him, amy had never forgotten the little boy that grew up next door. so knives and forks clinked on ceramic plates as our two figures both

KeyboardInterrupt: 

In [ ]:
pip install onnxruntime

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 59.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 4.9 MB/s eta 0:00:00


In [ ]:
import torch
import onnxruntime as ort
from sentence_transformers import SentenceTransformer
import numpy as np

# Step 1: Load the SentenceTransformer model
model = SentenceTransformer("all-MiniLM-L6-v2")
model.eval()  # Set the model to evaluation mode

# Step 2: Convert the SentenceTransformer model to ONNX format
dummy_input = {
    'input_ids': torch.ones(1, 10).long(),  # Simulate input IDs (dummy token IDs)
    'attention_mask': torch.ones(1, 10).long()  # Simulate attention mask
}

# Export the model to ONNX
torch.onnx.export(model,
                  (dummy_input['input_ids'], dummy_input['attention_mask']),  # Inputs to the model
                  "sentence_transformer.onnx",
                  opset_version=12,
                  input_names=['input_ids', 'attention_mask'],  # Model expects input_ids and attention_mask
                  output_names=['output'])

print("Model has been exported to ONNX format.")

TypeError: SentenceTransformer.forward() takes 2 positional arguments but 3 were given

In [ ]:
import torch
from sentence_transformers import SentenceTransformer

# Load the SentenceTransformer model (all-MiniLM-L6-v2)
model = SentenceTransformer("all-MiniLM-L6-v2")
model.eval()  # Set the model to evaluation mode

# Create dummy input tensors for input_ids and attention_mask
input_ids = torch.ones(1, 10).long()  # Simulated input_ids (dummy token IDs)
attention_mask = torch.ones(1, 10).long()  # Simulated attention mask

# Create a tuple of inputs (input_ids, attention_mask)
inputs = (input_ids, attention_mask)

# Export the model to ONNX format
torch.onnx.export(
    model[0],  # Access the actual transformer model (e.g., BERT or MiniLM)
    inputs,    # Pass the inputs as a tuple
    "all_miniLM_model.onnx",  # Export the model to ONNX format
    input_names=["input_ids", "attention_mask"],  # Input names for ONNX
    output_names=["output"],  # Output names for ONNX
    dynamic_axes={  # Allow dynamic batch size
        "input_ids": {0: "batch_size"},
        "attention_mask": {0: "batch_size"},
        "output": {0: "batch_size"},
    },
    opset_version=12  # ONNX opset version
)

print("Model has been successfully exported to ONNX format.")


TypeError: Transformer.forward() takes 2 positional arguments but 3 were given